Ship direction (angle in radians) is obviously a Markov chain, because,
if we fix a direction of the ship on a step $t$,
its next rotation (on step $t + 1$) will not change if another previous ($< t$) directions change,
and will depend only on step $t$.
It can be explained recursively

\begin{equation}
    \begin{cases}
        \eta_k &= \eta_{k - 1} + \xi_k, \\
        \eta_0 &= 0,
    \end{cases}
\end{equation}

where $\xi_k \sim N\left( 0, \frac{1}{4} \right)$.

Thus, position of the ship is a process that depends on the $\eta$

\begin{equation}
    \begin{cases}
        \zeta^x_k &= \zeta^x_{k - 1} - 15 \cdot \cos{\eta_k}, \\
        \zeta^y_k &= \zeta^y_{k - 1} + 15 \cdot \sin{\eta_k}, \\
        \zeta^x_0 &= 500, \\
        \zeta^y_0 &= 0.
    \end{cases}
\end{equation}

The task is to plot distributions of $\pmb{\zeta}_{12}$ and $\pmb{\zeta}_{24}$.